## Import libraries

In [1]:
import numpy as np
import cv2 as cv
import csv
import tensorflow as tf
from tensorflow.python.keras.models import load_model
import pafy

## Setting capture window parameters and loading labels

In [2]:
#Window Parameters.
frameWidth=640
frameHeight=480
brightness= 180

#Setting threshold limit(tunable).
threshold= 0.95

font= cv.FONT_HERSHEY_SIMPLEX

# Loading class labels into dictionary.
with open('labels.csv', mode='r') as infile:
    reader = csv.reader(infile)
    mydict = {rows[0]:rows[1] for rows in reader}      

## Loading our model

In [3]:
model=load_model("tsc_model.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## Preprocessing of frame

In [4]:

def preprocess(img):
      
    img = cv.resize(img,(32,32))
    
    #Converting to grayscale.
    img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    img = cv.equalizeHist(img)
    
    #Normalize image.
    img=img/255   
    
    # Reshaping image to (1,32,32,1).       
    img=img[np.newaxis,:,:,np.newaxis]
    return img

## Importing OpenVINO libraries for optimization of model

In [5]:
import keras2onnx
import onnx
import onnxruntime

## Converting model to ONNX format

In [6]:
# Convert to onnx model.
onnx_model = keras2onnx.convert_keras(model, model.name)
temp_model_file = 'tsc.onnx'

#Saving the model.
keras2onnx.save_model(onnx_model, temp_model_file)
sess = onnxruntime.InferenceSession(temp_model_file)

The ONNX operator number change on the optimization: 36 -> 19


## Importing Inference Engine libraries

In [7]:
from openvino.inference_engine import IECore
from openvino.inference_engine import IENetwork

## Using optimized model from OpenVINO to generate output

In [9]:
#Loading model for generating inference
def load_to_IE(model):
    
    ie = IECore()
    net = ie.read_network(model=r"C:\Program Files (x86)\Intel\openvino_2021.2.185\deployment_tools\model_optimizer\tsc.xml", weights=r"C:\Program Files (x86)\Intel\openvino_2021.2.185\deployment_tools\model_optimizer\tsc.bin")
    exec_net = ie.load_network(network=net, device_name="CPU")

    return exec_net


def do_inference(exec_net, image):
    
    input_blob = next(iter(exec_net.inputs))
    
    return exec_net.infer({input_blob: image})


# load our model
tsc_net = load_to_IE("tsc_model.h5")


# We need dynamically generated key for fetching output tensor
tsc_outputs = list(tsc_net.outputs.keys())

# Taking video input from camera.
#url= "https://www.youtube.com/watch?v=SWaYRyi0TTs"
#video= pafy.new(url)
#best = video.getbest(preftype="mp4")   
cap= cv.VideoCapture(0)
cap.set(3,frameWidth)
cap.set(4,frameHeight)
cap.set(10,brightness)
#cap.open(best.url)


while True:
    
    # If videocapture fails, exit else continue. 
    ret, image = cap.read()
    if not ret:
        break
    
    #Taking video frame and converting into numpy array.
    img = np.asarray(image)
    
    # Preprocessing frame by calling method.
    img = preprocess(img)
    
    # Formatting for output display text.
    cv.putText(image, "CLASS: ",(20,35),font,0.75,(0,0,255),2,cv.LINE_AA)
    cv.putText(image, "PROBABILITY: ",(20,75),font,0.75,(255,0,0),2,cv.LINE_AA)
    
    # Inference
    output = do_inference(tsc_net, image=img)
    
    # Storing label with maximum probability and probability score.
    classIndex = np.argmax(output[tsc_outputs[0]],axis=1)
    confidence = np.amax(output[tsc_outputs[0]])
    
    
    print(classIndex, confidence)
    
    # If probability score satisfies threshold limit, show output.
    if confidence>threshold:
        cv.putText(image, str(classIndex)+" "+mydict[str(max(classIndex))],(120,35),font,0.75,(0,0,255),2,cv.LINE_AA)
        cv.putText(image, str(round(confidence,2)),(180,75),font,0.75,(255,0,0),2,cv.LINE_AA)

    cv.imshow("Result", image)
    
    # Setting wait time.
    k = cv.waitKey(1) & 0xFF == ord('q')
    if k==27:
        break

cap.release()
cap.destroyAllWindows()



c:\users\avirup\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 'inputs' property of ExecutableNetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoCPtr objects which can be accessed by 'input_info' property.
  del sys.path[0]


[34] 0.5685355
[11] 0.63356614
[13] 0.69497603
[34] 0.99984705
[11] 0.37925526
[34] 0.9995147
[11] 0.93520224
[34] 0.832101
[11] 0.9444946
[13] 0.8395228
[34] 0.9967012
[13] 0.4138354
[34] 0.9981231
[13] 0.3783754
[34] 0.9187203
[35] 0.30287173
[13] 0.71997297
[11] 0.3772514
[13] 0.7451385
[34] 0.7077362
[15] 0.74131626
[34] 0.7637567
[13] 0.5205173
[34] 0.5286395
[13] 0.7757766
[13] 0.50485855
[3] 0.5528269
[13] 0.7483994
[34] 0.47866136
[15] 0.5973023
[34] 0.65185916
[15] 0.669979
[34] 0.6643148
[13] 0.36929235
[34] 0.506613
[3] 0.4094406
[13] 0.964893
[34] 0.47720706
[15] 0.58970326
[34] 0.7389219
[13] 0.3116197
[34] 0.79281175
[13] 0.40866563
[34] 0.85721797
[35] 0.40451995
[13] 0.6930796
[3] 0.6101982
[13] 0.6814619
[34] 0.6838663
[15] 0.43071213
[34] 0.9097236
[13] 0.56839234
[34] 0.9632952
[3] 0.40137294
[34] 0.70336944
[3] 0.7519499
[13] 0.71482205
[3] 0.45615083
[15] 0.4721969
[34] 0.8738638
[13] 0.3918635
[34] 0.9716377
[3] 0.38047472
[34] 0.9712017
[3] 0.7023043
[13] 0.71984

[11] 0.57976186
[15] 0.7149034
[35] 0.774571
[13] 0.8954623
[35] 0.56360805
[35] 0.94861853
[35] 0.8741673
[11] 0.9034157
[13] 0.46142837
[11] 0.50547117
[13] 0.7404011
[33] 0.37904868
[13] 0.90689784
[35] 0.4585553
[13] 0.9104222
[35] 0.85404414
[35] 0.7182875
[35] 0.92330563
[35] 0.4452023
[15] 0.36353603
[11] 0.46566084
[15] 0.97013277
[37] 0.66031975
[15] 0.63725007
[35] 0.5480337
[13] 0.7037732
[35] 0.89440125
[10] 0.501303
[35] 0.8172463
[11] 0.3354573
[15] 0.6978826
[11] 0.56858784
[15] 0.8837044
[33] 0.34759498
[13] 0.53397197
[35] 0.6494759
[15] 0.42835537
[35] 0.92543817
[35] 0.58783925
[35] 0.8077778
[11] 0.6026264
[15] 0.7458319
[11] 0.38074565
[15] 0.8121638
[33] 0.51139694
[15] 0.5094057
[35] 0.5878149
[15] 0.6319628
[35] 0.900501
[13] 0.5456454
[35] 0.714331
[33] 0.5361827
[15] 0.5728015
[11] 0.3869766
[13] 0.89956915
[37] 0.35161382
[15] 0.91301143
[35] 0.4200433
[15] 0.48190716
[35] 0.86755747
[13] 0.617663
[35] 0.5016949
[11] 0.6965653
[15] 0.7340555
[33] 0.7450345
[1

[8] 0.9999993
[8] 0.9999918
[8] 0.99999344
[8] 0.9985921
[0] 0.6754885
[0] 0.73973024
[0] 0.9641789
[0] 0.5084101
[0] 0.851806
[0] 0.9194928
[8] 0.99289715
[8] 0.9900379
[8] 0.99530506
[8] 0.6682769
[8] 0.76707035
[8] 0.9899762
[8] 0.698632
[0] 0.9994055
[0] 0.99987864
[0] 0.99987483
[0] 0.99517244
[0] 0.9831063
[8] 0.828667
[8] 0.7129337
[8] 0.977099
[8] 0.998774
[8] 0.95815766
[8] 0.9880988
[8] 0.9997092
[8] 0.9956577
[8] 0.9935988
[8] 0.9937162
[8] 0.99901354
[8] 0.99888283
[8] 0.9999094
[8] 0.99808323
[8] 0.99901736
[8] 0.98107404
[8] 0.99991393
[8] 0.9998871
[8] 0.99993324
[8] 0.99985254
[8] 0.99935204
[8] 0.9988412
[8] 0.9903867
[8] 0.9799575
[8] 0.9839771
[8] 0.9578231
[0] 0.852448
[8] 0.978554
[8] 0.9999865
[8] 0.9985613
[8] 0.99994934
[0] 0.59861594
[0] 0.92371726
[0] 0.99980146
[0] 0.99907047
[0] 0.99944776
[0] 0.9984738
[0] 0.976851
[8] 0.90666604
[8] 0.99999464
[8] 0.9999994
[8] 0.9999982
[8] 0.99999213
[8] 0.9999459
[8] 0.9999951
[8] 0.99999213
[8] 0.9999944
[8] 0.91419595

[0] 0.99392164
[0] 0.9883152
[0] 0.59941816
[8] 0.88032407
[18] 0.5445198
[11] 0.14303543
[15] 0.5349869
[29] 0.3774123
[18] 0.7794684
[34] 0.97779226
[34] 0.9994136
[29] 0.48033693
[29] 0.5643737
[22] 0.87883157
[40] 0.3724468
[34] 0.9037104
[11] 0.52818984
[13] 0.4986256
[35] 0.6381463
[25] 0.99920446
[35] 0.42323422
[11] 0.94017357
[34] 0.8446406
[11] 0.99584866
[13] 0.53091747
[11] 0.96664333
[15] 0.6132993
[11] 0.2617251
[13] 0.40917063
[34] 0.8264362
[3] 0.33475193
[34] 0.8865559
[13] 0.25797078
[35] 0.68148416
[11] 0.87110966
[13] 0.3666272
[11] 0.6038422
[13] 0.6085815
[34] 0.72487026
[35] 0.4410937
[34] 0.97466314
[13] 0.37486994
[35] 0.6683672
[11] 0.892753
[35] 0.7322175
[11] 0.9547767
[13] 0.94636464
[35] 0.6257882
[13] 0.8983791
[34] 0.9367847
[13] 0.50347626
[35] 0.64887494
[11] 0.5358935
[35] 0.8676239
[11] 0.82081443
[13] 0.8566391
[11] 0.48792464
[18] 0.372039
[34] 0.7434505
[13] 0.5447694
[34] 0.94290215
[33] 0.24610004
[34] 0.51740366
[11] 0.85828817
[35] 0.4288207
[

[37] 0.42807993
[13] 0.7363144
[13] 0.7859843
[13] 0.99997497
[13] 0.8727427
[13] 0.4804914
[34] 0.81164926
[35] 0.4587798
[34] 0.9904213
[37] 0.70682913
[34] 0.97356343
[37] 0.39918685
[13] 0.6104618
[11] 0.4278634
[13] 0.9743258
[35] 0.5352252
[1] 0.49180093
[34] 0.9993024
[37] 0.94573426
[34] 0.9999373
[35] 0.61369336
[34] 0.9998267
[35] 0.5963441
[13] 0.4979224
[34] 0.9350384
[1] 0.6999437
[34] 0.999767
[35] 0.8425639
[34] 0.99898535
[35] 0.85769856
[34] 0.9952225
[35] 0.85597074
[34] 0.86387527
[35] 0.62762463
[35] 0.75557685
[34] 0.94198614
[35] 0.6940959
[34] 0.9960194
[35] 0.66851175
[34] 0.9958075
[35] 0.8952827
[34] 0.99007857
[35] 0.76778185
[35] 0.38531488
[34] 0.6544821
[35] 0.7980468
[34] 0.99158514
[35] 0.5417793
[34] 0.9961761
[35] 0.91828465
[34] 0.9928819
[35] 0.72857165
[34] 0.6088358
[35] 0.6507975
[35] 0.96328485
[34] 0.970901
[24] 0.3769154
[34] 0.9987185
[35] 0.8238987
[34] 0.99934465
[35] 0.89938706
[34] 0.9960431
[13] 0.53263706
[13] 0.5143945
[34] 0.96674335
[

[34] 0.994344
[34] 0.33328432
[18] 0.5829821
[34] 0.94921565
[18] 0.4738784
[34] 0.993254
[18] 0.52535665
[34] 0.99842334
[35] 0.34206098
[34] 0.99129254
[18] 0.4312972
[18] 0.7604679
[34] 0.96165746
[15] 0.5748221
[34] 0.9978806
[18] 0.37408426
[34] 0.99419045
[34] 0.3282073
[34] 0.99556917
[18] 0.38395572
[18] 0.6550818
[34] 0.9798502
[18] 0.35529092
[34] 0.99794
[35] 0.5070453
[34] 0.9978957
[18] 0.47902405
[34] 0.6783723
[18] 0.56765234
[15] 0.60635215


KeyboardInterrupt: 